In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("..")

from xlstm_scaling_laws.analysis.mlstm_flop_count.flop_counting import (
    make_mlstm_fw_flop_comparison,
    make_mlstm_fwbw_flop_bw_apporox_comparison
)
from xlstm_scaling_laws.analysis.mlstm_flop_count.configuration import mlstm_model_config

import pandas as pd
# Set display option for scientific notation
pd.options.display.float_format = '{:.2e}'.format

# mLSTM Model FLOP analysis

Calculate the mLSTM FLOPs with different ways and compare to approximations.

## 1. Compare the two forward FLOP computations for mLSTM

"first": The first time I implemented the flops counts.
"tfla": More mature flop counting setup. 

-> Also check the impact of rounding the ffn dimension to multiple of 64.

**Result**: 
- There is only a slight deviation in the 4th digit. We use tlfa as new default.
- We also use rounding of the ffn dim to multiple of 64 the mLSTM FLOPs as default.

In [2]:
fw_df = make_mlstm_fw_flop_comparison(mlstm_model_config, include_embedding_flops=False, include_final_logit_flops=True)

In [3]:
styled_df = fw_df.style.format(
    "{:.4e}",
    subset=pd.IndexSlice[
        :, fw_df.select_dtypes(include=["float64", "float32"]).columns
    ],
)

In [4]:
styled_df

,model_name,#FLOPs_0_fw-mlstm_fw-first--causal_factor-1.0--round_ffn-False--bw-total_factor_2--incl_emb-False--incl_logit-True,#FLOPs_1_fw-mlstm_fw-tfla--causal_factor-0.75--round_ffn-False--bw-total_factor_2--incl_emb-False--incl_logit-True,#FLOPs_2_fw-mlstm_fw-first--causal_factor-1.0--round_ffn-True--bw-total_factor_2--incl_emb-False--incl_logit-True,#FLOPs_3_fw-mlstm_fw-tfla--causal_factor-1.0--round_ffn-True--bw-total_factor_2--incl_emb-False--incl_logit-True
0,mlstm_160M,2.0618e+12,2.0616e+12,2.0906e+12,2.0905e+12
1,mlstm_400M,5.9409e+12,5.9407e+12,5.9663e+12,5.9661e+12
2,mlstm_830M,1.2625e+13,1.2625e+13,1.2740e+13,1.2740e+13
3,mlstm_1.4B,2.1785e+13,2.1784e+13,2.1886e+13,2.1886e+13
4,mlstm_2.7B,4.4189e+13,4.4188e+13,4.4271e+13,4.4271e+13
5,mlstm_7B,1.1030e+14,1.1030e+14,1.1043e+14,1.1043e+14


## 2. Compare the Backward FLOP approximations for the mLSTM models

Outcome: Factor 2 approximation is the closest to the by hand backward flop computation.

In [5]:
fwbw_df = make_mlstm_fwbw_flop_bw_apporox_comparison(
    mlstm_model_config, include_embedding_flops=False, include_final_logit_flops=True
)
fwbw_styled_df = fwbw_df.style.format(
    "{:.4e}",
    subset=pd.IndexSlice[
        :, fwbw_df.select_dtypes(include=["float64", "float32"]).columns
    ],
)
fwbw_styled_df

TypeError: count_flops_mlstm_cell_chunkwise_bw__tfla() missing 2 required positional arguments: 'factor_sig' and 'factor_log'

## Compare the FLOP approximations from literature

We want to show that Kaplan and Distilling scaling law approximations to not apply to the mLSTM.

- -> It is a linear RNN and not Transformer with Atttention.
- -> We need a different approximation.